In [1]:
# import sys
import os
import requests
import re
# import warnings
# warnings.filterwarnings('ignore')
# relative import error with python modules and what not, the path needs to change according to cwd :)
# project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
# if project_root not in sys.path:
#     sys.path.append(project_root)

#### This notebook extracts GitHub URLS from the [MLBench](https://huggingface.co/datasets/super-dainiu/ml-bench) dataset, searches for any Arxiv links, and then downloads all the papers it has found (exactly 17 of the 18 repositories in the dataset contained links to Arxiv). It then creates one large embeddings database with all the papers.

In [1]:
# hf
from datasets import load_from_disk

# a2a
# from arxiv2arxode.utils.arxiv import arxiv_search
# import arxiv2arxode.lib.embeddings as emb

In [2]:
ds = load_from_disk("./datasets/")

In [13]:
len(ds['quarter'])+ len(ds['full']) + 315

643

In [9]:
len([d for d in ds['id_train'] if d['type']=="Python Code"])

488

In [ ]:
ds['quarter'][1]['github']

In [ ]:
unique_repos = set()
for subset in ds.values():
    unique_repos.update(subset['github'])
unique_repos, len(unique_repos)

In [ ]:

GITHUB_API = "https://api.github.com/repos/{}/readme"

ARXIV_REGEX = re.compile(r'arxiv.org/(abs|pdf)/\d{4}\.\d{5}')

def fetch_readme(repo_full_name, token=None):
    headers = {
        "X-GitHub-Api-Version": "2022-11-28"
    }
    if token:
        headers['Authorization'] = f'Bearer {token}'

    url = GITHUB_API.format(repo_full_name)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        readme_data = response.json()
        readme_content = requests.get(readme_data['download_url']).text
        return readme_content
    else:
        print(f"Failed to fetch README for {repo_full_name}: {response.status_code}")
        return None

def check_for_arxiv_links(readme_content):
    m = ARXIV_REGEX.search(readme_content)
    if m:
        url = m.group(0)
        return url

def d(repo_list, token=None):
    xs = []
    for repo in repo_list:
        readme_content = fetch_readme(repo, token)
        if readme_content:
            x = check_for_arxiv_links(readme_content)
            if x:
                print(x)
                xs.append(x)
            else:
                print(None)
    return xs

github_token = os.environ.get("GH_PAT")

In [ ]:
# "/".join(list(unique_repos)[0].split("/")[3:])

mlbench_repos = ["/".join(repo.split("/")[3:]) for repo in list(unique_repos)]
# mlbench_repos
arxiv_links = d(mlbench_repos, github_token)
# paperlist = d(["/".join(repo.split("/")[3:]) for repo in list(unique_repos)], github_token)


In [ ]:
# arxiv_links[0].split("/")[-1]
arxiv_ids = [link.split("/")[-1] for link in arxiv_links]
arxiv_ids

In [ ]:

# arxiv_search.ArxivScraper().download_papers(ids = arxiv_ids[:1], dirpath="./MLBench_papers/")
# arxiv_search.ArxivScraper().download_papers(ids = arxiv_ids[1:], dirpath="./MLBench_papers/")
for pid in arxiv_ids[2:]:
    arxiv_search.ArxivScraper().download_papers(ids = [pid], dirpath="./MLBench_papers/")
    print(f"Downloaded {pid}")

In [ ]:
chunked_documents = emb.load_and_chunk_papers(
    pdf_path="./MLBench_papers/",
)
# print(len(chunked_documents))
# chunked_documents[:10]


In [ ]:
# replace null char with unicode replacement char
for i in range(len(chunked_documents)):
    chunked_documents[i].page_content = chunked_documents[i].page_content.replace(
        "\x00", "\uFFFD"
    )

In [ ]:
collection_name = "MLBench_papers"
pgvec_connection = emb.create_embedding_collection(
    chunked_docs=chunked_documents,
    embeddings=emb.get_embedding_func(model="text-embedding-3-large"),
    collection_name=collection_name,
)


In [ ]:
pgvec_connection.similarity_search("kidney embryos", k=4)